# 整洁的远期收益率

## 安装软件包

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import cvxpy as cvx
import numpy as np
import pandas as pd
import time
import os
import quiz_helper
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

### 数据包

In [ ]:
import os
import quiz_helper
from zipline.data import bundles

In [ ]:
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..','data','module_4_quizzes_eod')
ingest_func = bundles.csvdir.csvdir_equities(['daily'], quiz_helper.EOD_BUNDLE_NAME)
bundles.register(quiz_helper.EOD_BUNDLE_NAME, ingest_func)
print('Data Registered')

### 构建管道引擎

In [ ]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar

universe = AverageDollarVolume(window_length=120).top(500) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(quiz_helper.EOD_BUNDLE_NAME)
engine = quiz_helper.build_pipeline_engine(bundle_data, trading_calendar)

### 查看数据

构建管道引擎后，我们获取时段结束时股票池中的股票。我们将使用这些 ticker 生成风险模型的收益率数据。

In [ ]:
universe_end_date = pd.Timestamp('2016-01-05', tz='UTC')

universe_tickers = engine\
    .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date)\
    .index.get_level_values(1)\
    .values.tolist()
    
universe_tickers

# 获取收益率数据

In [ ]:
from zipline.data.data_portal import DataPortal

data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

## 获取股价数据的辅助函数

In [ ]:
def get_pricing(data_portal, trading_calendar, assets, start_date, end_date, field='close'):
    end_dt = pd.Timestamp(end_date.strftime('%Y-%m-%d'), tz='UTC', offset='C')
    start_dt = pd.Timestamp(start_date.strftime('%Y-%m-%d'), tz='UTC', offset='C')

    end_loc = trading_calendar.closes.index.get_loc(end_dt)
    start_loc = trading_calendar.closes.index.get_loc(start_dt)

    return data_portal.get_history_window(
        assets=assets,
        end_dt=end_dt,
        bar_count=end_loc - start_loc,
        frequency='1d',
        field=field,
        data_frequency='daily')

## 将股价数据放入 dataframe 中

In [ ]:
returns_df = \
    get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        universe_end_date - pd.DateOffset(years=5),
        universe_end_date)\
    .pct_change()[1:].fillna(0) #convert prices into returns

returns_df

## 行业数据辅助函数

我们将为你创建一个对象，它会针对每支股票定义一个行业。行业由整数表示。我们继承了 Classifier 类。[Classifier 文档](https://www.quantopian.com/posts/pipeline-classifiers-are-here)以及 [Classifier 的源代码

In [ ]:
from zipline.pipeline.classifiers import Classifier
from zipline.utils.numpy_utils import int64_dtype
class Sector(Classifier):
    dtype = int64_dtype
    window_length = 0
    inputs = ()
    missing_value = -1

    def __init__(self):
        self.data = np.load('../../data/project_4_sector/data.npy')

    def _compute(self, arrays, dates, assets, mask):
        return np.where(
            mask,
            self.data[assets],
            self.missing_value,
        )

In [ ]:
sector = Sector()

## 我们将使用 2 年的数据计算因子

**注意：**2 年前的日期是休市日期。管道软件包不会处理开始或结束日期是休市的日期。为了解决这个问题，我们再往回推 2 天，这时候市场没有休市。

In [ ]:
factor_start_date = universe_end_date - pd.DateOffset(years=2, days=2)
factor_start_date

## 创建平滑的动量因子

In [ ]:
from zipline.pipeline.factors import Returns
from zipline.pipeline.factors import SimpleMovingAverage


# create a pipeline called p
p = Pipeline(screen=universe)
# create a factor of one year returns, deman by sector, then rank
factor = (
    Returns(window_length=252, mask=universe).
    demean(groupby=Sector()). #we use the custom Sector class that we reviewed earlier
    rank().
    zscore()
)


# Use this factor as input into SimpleMovingAverage, with a window length of 5
# Also rank and zscore (don't need to de-mean by sector, s)
factor_smoothed = (
    SimpleMovingAverage(inputs=[factor], window_length=5).
    rank().
    zscore()
)

# add the unsmoothed factor to the pipeline
p.add(factor, 'Momentum_Factor')
# add the smoothed factor to the pipeline too
p.add(factor_smoothed, 'Smoothed_Momentum_Factor')

## 可视化管道

注意，如果图片在 notebook 中加载很慢，请右击并在单独的标签页中查看图片。

In [ ]:
p.show_graph(format='png')

## 运行管道并查看因子数据

In [ ]:
df = engine.run_pipeline(p, factor_start_date, universe_end_date)

In [ ]:
df.head()

## 评估因子

我们将介绍评估 alpha 因子可以用到的一些工具。我们将使用 [alphalens 库](https://github.com/quantopian/alphalens)


## 导入 alphalens

In [ ]:
import alphalens as al

## 获取股价数据

注意，我们已经获取股价数据并转换为收益率，然后根据收益率计算因子。我们将再次获取股价数据，但是不会转换为收益率。这是因为我们将使用 alphalens 函数，这些函数的输入必须是股价，不是收益率。

## 定义资产列表

为了确保我们获取的是具有因子值的股票的股价，我们将获取资产列表，该列表可能是原始股票池的子集

In [ ]:
# get list of stocks in our portfolio (tickers that identify each stock)
assets = df.index.levels[1].values.tolist()
print(f"stock universe number of stocks {len(universe_tickers)}, and number of stocks for which we have factor values {len(assets)}")

In [ ]:
factor_start_date

In [ ]:
pricing = get_pricing(
        data_portal,
        trading_calendar,
        assets, #notice that we used assets instead of universe_tickers; in this example, they're the same
        factor_start_date, # notice we're using the same start and end dates for when we calculated the factor
        universe_end_date)

## 再次检查股价数据的日期

检查它们与因子数据相比，是否合理

In [ ]:
pricing.head(2)

In [ ]:
pricing.tail(2)

In [ ]:
pricing.shape

### 与一只股票的因子数据进行比较

In [ ]:
stock_index_name = df.index.get_level_values(1)[3] #just pick a stock; in this case, stock number 3 is AAPL
single_stock_factor_df = df[np.in1d(df.index.get_level_values(1), [stock_index_name])]

In [ ]:
single_stock_factor_df.head(2)

In [ ]:
single_stock_factor_df.tail(2)

In [ ]:
single_stock_factor_df.shape

## 小测验 1
如果你有在时间 t 前计算的因子值，你会使用什么股价数据来计算该数据的因子收益率？

## 答案 1：

## 为 alphalens 准备输入数据

Alphalens 会确保数据采用正确的格式，以便其他整洁的 alphalens 函数可以处理数据。例如，它会将股价数据与因子数据对齐，并计算与每个因子值相关的远期收益率。
我们将使用 [alphalens.utils.get_clean_factor_and_forward_returns](https://github.com/quantopian/alphalens/blob/master/alphalens/utils.py)

源代码描述了它的作用（我只展示了我们将使用的参数）：

In [ ]:
def get_clean_factor_and_forward_returns(factor,
                                         prices,
                                         ...
                                         periods=(1, 5, 10),
                                         ...
                                         ):

...

我们向其提供三个输入：factor、prices 和 periods。
* factor 是指包含每支股票在每一天的因子分数的序列。
* prices 是指每支股票在每一天的价格序列（日期与 factor 的一样）。注意，如果我们提供的时段大于 1，我们需要用时段量填充股价数据，从而计算远期收益率。请继续阅读“periods”参数的详细信息。
* periods：我们将针对该时段计算远期收益率。例如，如果股价和因子数据每天一个数据点（每日数据），并且我们希望每天计算因子加权投资组合的收益率，那么时段为 1，并且输入列表 [1]。如果我们想计算每周收益率，那么输入 [5]。如果我们想同时计算每日和每周收益率，那么输入 [1,5]。

* returns: 多索引 Pandas dataframe，其中包含数据的整洁版本。

## 小测验 2
get_clean_factor_and_forward_returns 会调用哪个 alphalens 函数来获得远期收益率？你可以在[源代码](https://github.com/quantopian/alphalens/blob/master/alphalens/utils.py)中找到答案

## 答案 2：


## 小测验 3
使用 alphalens 清理并对齐因子和远期收益率

即未平滑的因子和平滑后的因子

## 答案 3：

In [ ]:
factor_names = df.columns
print(f"The factor names are {factor_names}")

# Use a dictionary to store each dataframe, one for each factor and its associated forward returns
factor_data = {}
for factor_name in factor_names:
    print("Formatting factor data for: " + factor_name)
    # TODO: get clean factor and forward returns for each factor
    # Choose single period returns (daily returns)
    factor_data[factor_name] = # ...

## 检查清理后的数据

In [ ]:
cleaned_smooth_factor = factor_data[factor_names[1]]
cleaned_smooth_factor.head()

In [ ]:
cleaned_smooth_factor.tail()

## 小测验 4
你认为 '1D' 列代表什么？

## 答案 4：


## 解答 Notebook

[解答 notebook](clean_forward_returns_solution.ipynb)